Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
*Наименование вакансии
*Предлагаемую зарплату (отдельно мин. и отдельно макс. и отдельно валюта)
*Ссылку на саму вакансию        
*Сайт откуда собрана вакансия

По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [3]:
from bs4 import BeautifulSoup as bs
import requests
import re
import pandas as pd
import json

In [4]:
class hh():
    def __init__(self,s_key):
        self.mian_link ='https://lipetsk.hh.ru'
        self.s_key=s_key

    def get_vacancies(self):
        url = '/search/vacancy'
        params = {'area': '2', 'clusters=true': 'true', 'enable_snippets': 'true', 'text': self.s_key,
                  'only_with_salary': 'true', 'from': 'cluster_compensation',
                  'showClusters': 'true'}
        header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
            'Accept': '*/*'}
        response = requests.get(self.mian_link + url, headers=header, params=params)
        soup = bs(response.text, 'lxml')
        my_data = soup.find_all('div', {'class': "vacancy-serp-item"})
        my_list = []
        self.__get_info_from_page(my_data,my_list)
        page = soup.find('script', {'data-name': 'HH/Pager'})
        num_pages=json.loads(page['data-params'])['pagesCount']
        while num_pages > 1:
            params = {'area': '2', 'clusters=true': 'true', 'enable_snippets': 'true', 'text': self.s_key,
                      'only_with_salary': 'true', 'from': 'cluster_compensation',
                      'showClusters': 'true','page':str((num_pages-1))}
            response = requests.get(self.mian_link + url, headers=header, params=params)
            soup = bs(response.text, 'lxml')
            my_data = soup.find_all('div', {'class': "vacancy-serp-item"})
            self.__get_info_from_page(my_data, my_list)
            num_pages-=1
        return my_list

    def __get_info_from_page(self,my_data,my_list):
        for i in my_data:
            my_dict = {}
            my_dict['Позиция'] = i.find('a', {'class': 'HH-LinkModifier'}).getText()
            my_dict['Link'] = i.find('a', {'class': 'HH-LinkModifier'})['href']
            if i.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}):
                my_dict['Организация'] = i.find('a', {'data-qa': 'vacancy-serp__vacancy-employer'}).text
            else:
                my_dict['Организация'] = "No Data"
            if i.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}):
                salary = self.__get_salary(i.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'}).text)
            my_dict['Зарплата мин'] = salary['min']
            my_dict['Зарплата макс'] = salary['max']
            my_dict['Валюта'] = salary['currency']
            my_dict['Ресурс'] = self.mian_link
            my_list.append(my_dict)

    def __get_salary(self,str):
          salary = {}
          if str[:2] == 'от':
              min = re.search(r'[\d\s]+', str)[0]
              min = int(re.sub(r"\s", '', min))
              max = None
          elif str[:2] == 'до':
              max = re.search(r'[\d\s]+', str)[0]
              max = int(re.sub(r"\s", '', max))
              min = None
          else:
              min = re.search(r'[\d\s]+', str)[0]
              min = int(re.sub(r"\s", '', min))
              max = re.search(r'-([\d\s]+)', str)[1]
              max = int(re.sub(r"\s", '', max))
          currency = re.search(r'[A-Zа-я]{3}', str)[0]
          salary['min'] = min
          salary['max'] = max
          salary['currency'] = currency
          return salary

In [21]:
class sj():
    def __init__(self,s_key):
        self.mian_link ='https://superjob.ru'
        self.s_key=s_key

    def get_vacancies_from_with_salary(self):
        url = '/vacancy/search'
        params = { 'keywords': self.s_key, 'payment_defined':'1'}
        header = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36',
            'Accept': '*/*'}
        response = requests.get(self.mian_link + url, headers=header, params=params)
        soup = bs(response.text, 'lxml')
        my_data = soup.find_all('div', {'class': "iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL"})
        my_list = []
        self.__get_info_from_page(my_data, my_list)
        while soup.find('a', {'rel': 'next'}):
            page=soup.find('a', {'rel': 'next'})['href']
            params = { 'keywords': self.s_key, 'payment_defined':'1'}
            response = requests.get(self.mian_link +page, headers=header, params=params)
            soup = bs(response.text, 'lxml')
            my_data = soup.find_all('div', {'class': "iJCa5 f-test-vacancy-item _1fma_ _1JhPh _2gFpt _1znz6 _2nteL"})
            self.__get_info_from_page(my_data, my_list)
        return my_list

    def __get_info_from_page(self, my_data, my_list):
        for i in my_data:
            my_dict = {}
            my_dict['Позиция'] = i.find('a').getText()
            my_dict['Link'] = self.mian_link+i.find('a')['href']
            my_dict['Организация'] = i.find('a', {'class':'_25-u7'}).text
            sal=i.find('span', {'class': 'PlM3e'}).text
            salary = self.__get_salary(sal)
            my_dict['Зарплата мин'] = salary['min']
            my_dict['Зарплата макс'] = salary['max']
            my_dict['Валюта'] = salary['currency']
            my_dict['Ресурс'] = self.mian_link
            my_list.append(my_dict)
        # print(my_dict)

    def __get_salary(self, str):
        salary = {}
        if str[:2] == 'от':
            min = re.search(r'[\d\s]+', str)[0]
            min = int(re.sub(r"\s", '', min))
            max = None
            currency = re.search(r'[A-Zа-я]{3}', str)[0]
        elif str[:2] == 'до':
            max = re.search(r'[\d\s]+', str)[0]
            max = int(re.sub(r"\s", '', max))
            min = None
            currency = re.search(r'[A-Zа-я]{3}', str)[0]
        elif '-' in str:
            min = re.search(r'[\d\s]+', str)[0]
            min = int(re.sub(r"\s", '', min))
            max = re.search(r'-([\d\s]+)', str)[1]
            max = int(re.sub(r"\s", '', max))
            currency = re.search(r'[A-Zа-я]{3}', str)[0]
        elif str=="По договорённости":
            min=None
            max=None
            currency = None
        else:
            min = re.search(r'[\d\s]+', str)[0]
            min = int(re.sub(r"\s", '', min))
            max = re.search(r'[\d\s]+', str)[0]
            max = int(re.sub(r"\s", '', max))
            currency = re.search(r'[A-Zа-я]{3}', str)[0]
        salary['min'] = min
        salary['max'] = max
        salary['currency'] = currency
        return salary

In [28]:
vac_hh=hh('Pithon')
vac_sj = sj('Pithon')

In [29]:
df = pd.DataFrame(vac_hh.get_vacancies()+vac_sj.get_vacancies_from_with_salary())

In [30]:
df

,Позиция,Link,Организация,Зарплата мин,Зарплата макс,Валюта,Ресурс
0,Senior Python Developer (backend),https://lipetsk.hh.ru/vacancy/37765447?query=P...,Интеллектуальные системы,250000.0,NaN,руб,https://lipetsk.hh.ru
1,Senior Machine Learning Engineer,https://lipetsk.hh.ru/vacancy/37703126?query=P...,ООО Giant,250000.0,450000.0,руб,https://lipetsk.hh.ru
2,Back-End Python разработчик (Middle/Senior),https://lipetsk.hh.ru/vacancy/38019682?query=P...,ООО Вайт Код,160000.0,220000.0,руб,https://lipetsk.hh.ru
3,Сетевой инженер,https://lipetsk.hh.ru/vacancy/37001116?query=P...,ООО Айфьючер,120000.0,NaN,руб,https://lipetsk.hh.ru
4,Ведущий специалист по разработке VoIP решений,https://lipetsk.hh.ru/vacancy/37724082?query=P...,Смарт Телеком,80000.0,NaN,руб,https://lipetsk.hh.ru
...,...,...,...,...,...,...,...
265,Инженер по качеству (для людей с ограниченными...,https://spb.superjob.ru/vakansii/inzhener-po-k...,Кадровое решение СПб,70000.0,NaN,руб,https://spb.superjob.ru
266,Ассистент/доцент кафедры информатики и компьют...,https://spb.superjob.ru/vakansii/assistent-113...,"Горный университет, Кафедра информатики и комп...",80000.0,80000.0,руб,https://spb.superjob.ru
267,Инженер встроенных систем / Linux Embedded,https://spb.superjob.ru/vakansii/inzhener-vstr...,Группа компаний MVS,150000.0,NaN,руб,https://spb.superjob.ru
268,Системный администратор,https://spb.superjob.ru/vakansii/sistemnyj-adm...,ГАРАНТ,NaN,35000.0,руб,https://spb.superjob.ru
